In [ ]:
!pip install catboost
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn
!pip install scipy

In [2]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, confusion_matrix, f1_score, classification_report, recall_score, mean_absolute_percentage_error
# from sklearn import preprocessing

import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np


In [5]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Программирование и основы алгоритмизации 1 новый.csv", delimiter=";", encoding='cp1251', decimal=",") #delimiter=";",
# data = data.dropna()
# print(len(data))

# not_for_prediction = ["ФИО", "Команда", "Направление", "Оценка", "Баллы", "Повышение оценки"]
# data = data[data.loc[:,:].columns.difference(not_for_prediction)]
data.drop_duplicates(keep='last')

# cols = data.select_dtypes(exclude=['float']).columns
# data[cols] = data[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
# columns = list(data.columns[pd.Series(data.columns).str.startswith('Посещение ')])
# data[data.loc[:,:].columns.difference(["Пол", "Не сдал(-а)"]+columns)] = data[data.loc[:,:].columns.difference(["Пол", "Не сдал(-а)"]+columns)].apply(lambda x: float(x)/100)
# data = data.round(4)
# z = np.abs(stats.zscore(data))
# data = data[(z<3).all(axis=1)]

ex = "Контрольная работа 1"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
exams = [name  for name in data.columns.to_list() if name.startswith("Контрольная")]
scores = [name  for name in data.columns.to_list() if name.startswith("Баллы ")]
being = [name  for name in data.columns.to_list() if name.startswith("Посещение ")]
before = [name  for name in data.columns.to_list() if name.startswith("Процент ")]
data = data.drop(data.columns.difference(["Не сдал(-а)", "Пол"]+exams+scores+being+before), axis=1)
def MakeFloat(d):
    convert_dict = {}
    for i, lst in enumerate([exams, scores, being, before]):
        for name in lst:
            convert_dict[name] = float
    convert_dict["Пол"] = float

    d = d.astype(convert_dict)
    return d
data = MakeFloat(data)
data.dtypes
def Mod100(d):
    d[exams+scores] = d[exams+scores].div(100)
    return d
data = Mod100(data)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
columns = list(data.columns[pd.Series(data.columns).str.startswith('Баллы до')])

# Iterate through the five airlines
for column in columns:
    display(data.boxplot(column=columns))
    break
    # Subset to the airline
#     subset = data[column]

#     # Draw the density plot
#     sns.distplot(subset, hist = True, kde = True,
#                  kde_kws = {'linewidth': 1},
#                  label = column)

# # Plot formatting
# plt.legend(prop={'size': 8}, title = 'Airline')

In [ ]:
# загрузка данных
import tkinter.filedialog as tkFileDialog
def open_file():
    return tkFileDialog.askopenfilename( title='Выберите файл:')
data = pd.read_csv(open_file(), delimiter=";", encoding='cp1251', decimal=",") #delimiter=";",
data = data.dropna()
convert_dict = { "Баллы": float }
data = data.astype(convert_dict)
print(len(data))

In [12]:
#  найти параметры регрессии
#найти наибольшую глубину
X = data[data.loc[:,:ex].columns.difference([ex, "Не сдал(-а)"]+before)] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
Y = data[ex].apply(lambda x: x*100).to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

estimator=AdaBoostRegressor()
# applying GridSearchCV
parameters_RF = {
     'n_estimators': range(1, 15),
    'learning_rate' : [0.01, 0.05, 0.1, 0.5],
    'loss' : ['linear', 'square', 'exponential'],
}
grid_search=GridSearchCV(cv=5, estimator=estimator, n_jobs=-1,
             param_grid=parameters_RF,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"AdaBoostRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )

estimator=RandomForestRegressor()
# applying GridSearchCV
parameters_RF = {'bootstrap': [True], 'max_depth': range(1, 15),
                'max_features': ['sqrt', 'log2', None],
                'min_samples_leaf': [1,2,3],
                'min_samples_split': [1,2,3],
                'n_estimators': range(1, 15)}
grid_search=GridSearchCV(cv=5, estimator=estimator, n_jobs=-1,
             param_grid=parameters_RF,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"RandomForestRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )


# estimator=DecisionTreeRegressor()
# # applying GridSearchCV
# parameters_RF={'criterion': ['squared_error', 'friedman_mse',
#                                        'absolute_error', 'poisson'],
#                          'max_depth': range(1, 15),
#                          'max_features': ['sqrt', 'log2', None],
#                          'min_samples_leaf': [1,2,3],
#                          'min_samples_split': [1,2,3],
#                          'splitter': ['best', 'random']}
# grid_search=GridSearchCV(cv=5, estimator=estimator, n_jobs=-1,
#              param_grid=parameters_RF,
#              scoring='neg_mean_absolute_error', verbose=2)
# pred=grid_search.fit(X_train, y_train)
# y_pred = pred.predict(X_test)
# print(f"DecisionTreeRegressor parameters: {grid_search.best_params_}" )
# print('Best Score:', grid_search.best_score_ )


estimator=KNeighborsRegressor()
# applying GridSearchCV
parameters_RF = {
    'n_neighbors': range(1,16),
    'leaf_size': range(20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}
grid_search=GridSearchCV(cv=5, estimator=estimator, n_jobs=-1,
             param_grid=parameters_RF,
             scoring='neg_mean_absolute_error', verbose=2)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"KNeighborsRegressor parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )


Fitting 5 folds for each of 168 candidates, totalling 840 fits
AdaBoostRegressor parameters: {'learning_rate': 0.1, 'loss': 'linear', 'n_estimators': 8}
Best Score: -3.2450782658462836
Fitting 5 folds for each of 5292 candidates, totalling 26460 fits
RandomForestRegressor parameters: {'bootstrap': True, 'max_depth': 3, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 14}
Best Score: -3.233075117299456
Fitting 5 folds for each of 2400 candidates, totalling 12000 fits
KNeighborsRegressor parameters: {'leaf_size': 34, 'metric': 'minkowski', 'n_neighbors': 15, 'p': 1, 'weights': 'uniform'}
Best Score: -3.449345170444076


In [14]:
#найти наибольшую глубину
X = data[data.loc[:,:ex].columns.difference(["Не сдал(-а)"])] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
Y = data["Не сдал(-а)"].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

estimator=AdaBoostClassifier()
# applying GridSearchCV
parameters_RF = {
    'n_estimators':range(1,50),
    'base_estimator':[None, DecisionTreeClassifier(), SVC(), LogisticRegression()],
    'learning_rate': [0.5, 1.0, 2.0, ]
}
grid_search=GridSearchCV(
    estimator=estimator,
    param_grid=parameters_RF)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"AdaBoostClassifier parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )

estimator = RandomForestClassifier()
parameters_RF={'max_depth':(1,15, 1),
              'n_estimators':[10,100,200],
              'max_features':[1,3,5,7],
              'min_samples_leaf':[1,2,3],
              'min_samples_split':[1,2,3]
           }
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_RF,
    scoring ='f1_micro',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred = pred.predict(X_test)
print(f"RandomForestClassifier parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )


estimator = KNeighborsClassifier(algorithm='auto')
parameters_KNN = {
    'n_neighbors': range(1,16),
    'leaf_size': range(20,40,1),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_KNN,
    scoring='f1_micro',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred_KNN1 =pred.predict(X_test)
print(f"Knn best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )


AdaBoostClassifier parameters: {'base_estimator': LogisticRegression(), 'learning_rate': 2.0, 'n_estimators': 48}
Best Score: 0.8705140961857379
RandomForestClassifier parameters: {'max_depth': 15, 'max_features': 1, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 100}
Best Score: 0.879436152570481
Knn best parameters: {'leaf_size': 20, 'metric': 'minkowski', 'n_neighbors': 12, 'p': 2, 'weights': 'uniform'}
Best Score: 0.8809508015478166


In [ ]:
models = {
    # "Метод опорных векторов": SVC(kernel='rbf'), \
            "k-ближайших соседей": KNeighborsClassifier(leaf_size=20, metric='chebyshev', n_neighbors=7, p=1, weights='uniform'), \
            # "Линейный дискриминантный анализ": LinearDiscriminantAnalysis(), \
            # "Наивный метод Байеса": GaussianNB(),  \
            "Рандомный лес": RandomForestClassifier(max_depth=15, max_features=1, min_samples_leaf=2, min_samples_split=3, n_estimators=10),  \
            # "Логистическая регрессия": LogisticRegression(),
            "CatBoost": CatBoostClassifier(iterations=21, learning_rate=0.4, silent=True, allow_writing_files=False),
            "AdaBoostClassifier": AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=2.0, n_estimators=2)}
models_regres = {"k-ближайших соседей": KNeighborsRegressor(leaf_size=29, metric='minkowski', n_neighbors=10, p=1, weights='uniform'),
            "Рандомный лес": RandomForestRegressor(bootstrap=True, max_depth=8, max_features=None, min_samples_leaf=3, min_samples_split=2, n_estimators=10),  \
            "CatBoostRegressor": CatBoostRegressor(iterations=151, learning_rate=0.2, depth=7, silent=True, allow_writing_files=False),
            "AdaBoostRegressor": AdaBoostRegressor(learning_rate=0.1, loss='exponential', n_estimators=5)}

In [16]:
X = data[data.loc[:,:ex].columns.difference(["Не сдал(-а)"])] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
Y = data["Не сдал(-а)"].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

estimator = CatBoostClassifier()
parameters_KNN = {
    'iterations': range(1,251, 10),
    'silent': [True],
    'allow_writing_files': [False],
    'learning_rate': [0.01, 0.05, 0.1, 0.5]
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_KNN,
    scoring='f1_micro',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred_KNN1 =pred.predict(X_test)
print(f"CatBoostClassifier best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )

X = data[data.loc[:,:ex].columns.difference([ex, "Не сдал(-а)"]+before)] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
Y = data[ex].apply(lambda x: x*100).to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

estimator = CatBoostRegressor()
parameters_KNN = {
    'iterations': range(1,251, 10),
    'silent': [True],
    'depth': range(1, 20),
    'allow_writing_files': [False],
    'learning_rate': [0.01, 0.05, 0.1, 0.5]
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_KNN,
    scoring='neg_mean_absolute_error',
    n_jobs = -1,
    cv = 5
)
pred=grid_search.fit(X_train, y_train)
y_pred_KNN1 =pred.predict(X_test)
print(f"CatBoostRegressor best parameters: {grid_search.best_params_}" )
print('Best Score:', grid_search.best_score_ )

CatBoostClassifier best parameters: {'allow_writing_files': False, 'iterations': 31, 'learning_rate': 0.1, 'silent': True}
Best Score: 0.8868767274737424
CatBoostRegressor best parameters: {'allow_writing_files': False, 'depth': 2, 'iterations': 61, 'learning_rate': 0.1, 'silent': True}
Best Score: -3.221443807325445


In [ ]:
# X = data[data.loc[:,:ex].columns.difference(not_for_prediction)] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
# Y = data["Не сдал(-а)"].to_list()
# learning_rate_list = [0.01]+[p/100.0 for p in range(5, 50, 5)]
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)
# max_acc = 0.0
# max_i = 0
# max_r = 0
# for i in range (1, 251, 5):
#     for r in learning_rate_list:
#         dtr = CatBoostClassifier(iterations=i, learning_rate=r, silent=True, allow_writing_files=False)
#         dtr.fit(X_train, y_train)
#         data_prediction = dtr.predict(X_test)

#         score = round(f1_score(y_test, data_prediction), 3)
#         if max_acc < score:
#             max_acc = score
#             max_r = r
#             max_i=i
# # Max acc: 0.952, iterations=21, learning_rate=0.4
# print(f"Max acc: {max_acc}, iterations={max_i}, learning_rate={max_r}")

X = data[data.loc[:,:ex].columns.difference(not_for_prediction+[ex])] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
Y = data[ex].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=27)

max_acc = 10000
max_i = 0
max_r = 0
max_d = 0
for i in range (1, 251, 25):
    for r in [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5]:
        for d in range(1, 16):
            dtr = CatBoostRegressor(iterations=i, depth=d, learning_rate=r, silent=True, allow_writing_files=False)
            dtr.fit(X_train, y_train)
            data_prediction = dtr.predict(X_test)

            score = round(mean_absolute_error(y_test, data_prediction), 3)
            if max_acc > score:
                max_acc = score
                max_r = r
                max_i = i
                max_d = d
print(f"Max acc: {max_acc}, iterations={max_i}, learning_rate={max_r}, depth={max_d}")

Max acc: 3.551, iterations=151, learning_rate=0.2, depth=7


In [ ]:
def prediction_regres(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=27)
    res = {}

    for key, model in models_regres.items():
        try:
        # print(X_train, y_train)
            model.fit(X_train, np.array(y_train))
            data_prediction = model.predict(X_test)

            res[key] =round(mean_absolute_error(list(y_test), list(data_prediction)), 3)
            print(key)
            try:
                feature_importances = pd.DataFrame(model.feature_importances_, index=X.columns, columns=['importance'])
                feature_importances = feature_importances.sort_values(by='importance', ascending=False)

                print(feature_importances)
            except:
                print("Impossible to compute features")
            #print(pd.DataFrame(data={"Настоящее": list(y_test)[:10], "Прогноз": list(data_prediction)[:10]}))
            # dfd = pd.DataFrame(data=[list(y_test), list(data_prediction)])
            # display(dfd)
            # print(key)

        except Exception as e:
            res[key] = 0
    return res


def prediction_class(X, Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=27)
    res = {}
    for key, model in models.items():
        model.fit(X_train, y_train)
        data_prediction = model.predict(X_test)
        res[key] = round(f1_score(data_prediction, y_test), 3)
        print(key, classification_report(y_test, data_prediction), confusion_matrix(y_test, data_prediction))
        try:
            feature_importances = pd.DataFrame(model.feature_importances_, index=X.columns, columns=['importance'])
            feature_importances = feature_importances.sort_values(by='importance', ascending=False)

            print(feature_importances)
        except:
            print("Impossible to compute features")
        data_proba = model.predict_proba(X_test)

        print(data_proba[:10])

        #print(pd.DataFrame(data={"Настоящее": list(y_test)[:10], "Прогноз": list(data_prediction)[:10]}))

        # print(accuracy_score(data_prediction, y_test))
    return res

In [ ]:
exams = [name  for name in data.columns.to_list() if name.startswith("Контрольная")]
scores = [name  for name in data.columns.to_list() if name.startswith("Баллы ")]
being = [name  for name in data.columns.to_list() if name.startswith("Посещение ")]

convert_dict = {name: float for name in being+scores+exams+["Не сдал(-а)"]}
# convert_dict["Не сдал(-а)"] = int
data = data.astype(convert_dict)

not_for_prediction = ["ФИО", "Команда", "Направление", "Не сдал(-а)", "Оценка", "Баллы", "Повышение оценки"] #, "Пол"

In [ ]:
# определение оценки
data["Оценка"] = 0
data.loc[data["Баллы"]<61.0, "Оценка"] = 2
data.loc[data["Баллы"]>=91.0, "Оценка"] = 5
data.loc[(data["Баллы"]<91) & (data["Баллы"]>=76.0), "Оценка"] = 4
data.loc[(data["Баллы"]<76.0) & (data["Баллы"]>=61.0), "Оценка"] = 3

data.head

<bound method NDFrame.head of                                   ФИО  Пол       Команда  Баллы  Сдал(-а)  \
0             Павлова Мария Сергеевна  0.0  АЛГЕБРА Л-08  13.75       0.0   
1         Глазков Владислав Артемович  1.0  АЛГЕБРА Л-03  63.00       1.0   
2             Ахмедов Анзор Насирович  1.0  АЛГЕБРА Л-03  61.50       1.0   
3    Шапошникова Анастасия Валерьевна  0.0  АЛГЕБРА Л-09  60.00       0.0   
4          Машаров Антон Владимирович  1.0  АЛГЕБРА Л-01  65.50       1.0   
..                                ...  ...           ...    ...       ...   
851     Дубровский Евгений Витальевич  1.0  АЛГЕБРА Л-06  61.00       1.0   
852       Хворова Ангелина Викторовна  0.0  АЛГЕБРА Л-06  96.00       1.0   
853             Бабин Никита Павлович  1.0  АЛГЕБРА Л-02  92.00       1.0   
854           Гумеров Радик Рамилевич  1.0  АЛГЕБРА Л-01  86.00       1.0   
855           Гаибов Мансур Маратович  1.0  АЛГЕБРА Л-05  52.00       0.0   

     Посещение до Контрольная работа 1  Баллы до Контрольная работа 1  \
0                                 0.55                           0.00   
1                                 1.00                           6.00   
2                                 0.82                           0.00   
3                                 0.73                           0.00   
4                                 0.27                           0.00   
..                                 ...                            ...   
851                               0.18                           5.75   
852                               1.00                           7.00   
853                               0.82                           0.00   
854                               0.64                           0.00   
855                               0.73                           0.00   

     Контрольная работа 1  Посещение до Контрольная работа 2  \
0                     9.0                                1.0   
1                     6.0                                1.0   
2                    11.0                                1.0   
3                    15.0                                1.0   
4                    18.0                                0.4   
..                    ...                                ...   
851                   4.0                                0.6   
852                  20.0                                1.0   
853                  20.0                                1.0   
854                  20.0                                1.0   
855                  11.0                                1.0   

     Баллы до Контрольная работа 2  Контрольная работа 2  \
0                              0.0                   0.0   
1                              7.5                   4.0   
2                              0.0                  18.0   
3                              0.0                  20.0   
4                              0.0                  14.5   
..                             ...                   ...   
851                            0.5                   3.2   
852                            4.0                  13.0   
853                           10.0                  15.0   
854                            0.0                  15.0   
855                            0.0                  15.0   

     Посещение до Контрольная работа 3  Баллы до Контрольная работа 3  \
0                                 0.38                           0.00   
1                                 1.00                          10.00   
2                                 1.00                           0.00   
3                                 1.00                           0.00   
4                                 0.62                          12.00   
..                                 ...                            ...   
851                               0.88                           8.55   
852                               1.00                           0.00   
853                  

In [ ]:
# повысил ли студент оценку на экзамене
data["Повышение оценки"] = 0
for index, row in data.iterrows():
    res = row[scores+exams].sum()
    en = row["Баллы"]
    # row["Повышение оценки"] = -1 if res<row["Баллы"] else 1 if res>row["Баллы"] else 0
    data.loc[index, "Повышение оценки"] = -1 if res<row["Баллы"] else 1 if res>row["Баллы"] else 0

print(f"Повысили баллы на экзамене {round(len(data.loc[data['Повышение оценки']==1, 'Баллы'])/len(data)*100, 2)}%")
print(f"Понизили баллы на экзамене {round(len(data.loc[data['Повышение оценки']==-1, 'Баллы'])/len(data)*100, 2)}%")
print(f"Не изменили баллы на экзамене {round(len(data.loc[data['Повышение оценки']==0, 'Баллы'])/len(data)*100, 2)}%")


Повысили баллы на экзамене 0.0%
Понизили баллы на экзамене 85.86%
Не изменили баллы на экзамене 14.14%


In [ ]:
# try:
# data.drop(["ФИО", "Команда"], axis=1, inplace=True)
print("Прогнозирование сдачи дисциплины")
results = pd.DataFrame(index=list(models.keys()), columns=exams)
for ex in exams:
    print(ex)
    results[ex] = ""
    columns = data.loc[:,:ex]
    columns =columns.columns.difference(not_for_prediction).to_list()

    temp_data = data[columns+["Не сдал(-а)"]]
    temp_data.drop_duplicates(keep='last')
    cols = temp_data.select_dtypes(exclude=['float']).columns
    temp_data[cols] = temp_data[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
    X = temp_data[temp_data.columns.difference(["Не сдал(-а)"])] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
    # X[X.loc[:,:].columns.difference(["Пол"])] = X[X.loc[:,:].columns.difference(["Пол"])].div(100)

    Y = list(temp_data["Не сдал(-а)"])

    pred_res = prediction_class(X, Y)
    for key, val in pred_res.items():
        results[ex][key] = val

results
# except:
#     # data.drop(["Направление"], axis=1, inplace=True)
#     X = data[data.columns.difference(["Направление", "Не сдал(-а)"])]
#     Y = data["Не сдал(-а)"].to_list()

Прогнозирование сдачи дисциплины
Контрольная работа 1
k-ближайших соседей               precision    recall  f1-score   support

         0.0       0.71      0.66      0.68        83
         1.0       0.87      0.89      0.88       203

    accuracy                           0.82       286
   macro avg       0.79      0.77      0.78       286
weighted avg       0.82      0.82      0.82       286
 [[ 55  28]
 [ 23 180]]
Impossible to compute features
[[0.         1.        ]
 [0.57142857 0.42857143]
 [0.42857143 0.57142857]
 [0.14285714 0.85714286]
 [0.42857143 0.57142857]
 [0.14285714 0.85714286]
 [0.28571429 0.71428571]
 [0.71428571 0.28571429]
 [0.85714286 0.14285714]
 [0.         1.        ]]
Рандомный лес               precision    recall  f1-score   support

         0.0       0.68      0.65      0.66        83
         1.0       0.86      0.87      0.87       203

    accuracy                           0.81       286
   macro avg       0.77      0.76      0.76       286
weighted

,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.876,0.875,0.929
Рандомный лес,0.866,0.897,0.934
CatBoost,0.863,0.905,0.934
AdaBoostClassifier,0.864,0.849,0.935


In [ ]:
# прогнозирование контрольных без учета оценок
# try:
# data.drop(["ФИО", "Команда"], axis=1, inplace=True)
print("Прогнозирование контрольных без учета оценок")
results = pd.DataFrame(index=list(models_regres.keys()), columns=exams)
for ex in exams:
    print(ex)
    results[ex] = ""
    columns = data.loc[:,:ex]
    columns =columns.columns.difference(not_for_prediction).to_list()

    temp_data = data[columns]
    temp_data.drop_duplicates(keep='last')
    cols = temp_data.select_dtypes(exclude=['float']).columns
    temp_data[cols] = temp_data[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
    X = temp_data[temp_data.columns.difference([ex, "Не сдал(-а)"])] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
    #X[X.loc[:,:].columns.difference(["Пол"])] = X[X.loc[:,:].columns.difference(["Пол"])].div(100)

    Y = list(temp_data[ex])
    pred_res = prediction_regres(X, Y)
    for key, val in pred_res.items():
        results[ex][key] = val #*100
display(results)
# except:
#     # data.drop(["Направление"], axis=1, inplace=True)
#     X = data[data.columns.difference(["Направление", "Не сдал(-а)"])]
#     Y = data["Не сдал(-а)"].to_list()

Прогнозирование контрольных без учета оценок
Контрольная работа 1
k-ближайших соседей
Impossible to compute features
Рандомный лес
                                   importance
Баллы до Контрольная работа 1        0.507721
Посещение до Контрольная работа 1    0.412726
Пол                                  0.079553
CatBoostRegressor
                                   importance
Баллы до Контрольная работа 1       51.839795
Посещение до Контрольная работа 1   37.677608
Пол                                 10.482596
AdaBoostRegressor
                                   importance
Баллы до Контрольная работа 1        0.588196
Посещение до Контрольная работа 1    0.403398
Пол                                  0.008406
Контрольная работа 2
k-ближайших соседей
Impossible to compute features
Рандомный лес
                                   importance
Контрольная работа 1                 0.477665
Баллы до Контрольная работа 2        0.137812
Баллы до Контрольная работа 1        0.132583
Посещение д

,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,5.542,4.238,3.723
Рандомный лес,5.544,4.144,3.748
CatBoostRegressor,5.227,4.125,3.898
AdaBoostRegressor,5.663,4.64,4.23


In [ ]:
# прогнозирование контрольных учитывая оценку
# try:
# data.drop(["ФИО", "Команда"], axis=1, inplace=True)

print("Прогнозирование контрольных с учетом оценок")
for name, group in data.groupby("Оценка"):
    results = pd.DataFrame(index=list(models_regres.keys()), columns=exams)
    for ex in exams:
        results[ex] = ""

        X = group[group.loc[:,:ex].columns.difference(not_for_prediction+[ex])] #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
        Y = group[ex].to_list()

        pred_res = prediction_regres(X, Y)
        for key, val in pred_res.items():
            results[ex][key] = val

    print(name)
    display(results)
# except:
#     # data.drop(["Направление"], axis=1, inplace=True)
#     X = data[data.columns.difference(["Направление", "Не сдал(-а)"])]
#     Y = data["Не сдал(-а)"].to_list()

In [ ]:
def div_100(d):
    cols = d[d.loc[:,:].columns.difference(not_for_prediction)].select_dtypes(exclude=['float']).columns
    d[cols] = d[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
    columns = list(d.columns[pd.Series(d.columns).str.startswith('Посещение ')])
    d[d.loc[:,:].columns.difference(["Пол", "Не сдал(-а)"]+columns+not_for_prediction)] = d[d.loc[:,:].columns.difference(["Пол", "Не сдал(-а)"]+columns+not_for_prediction)].div(100)
    return d.round(4)

        # print(accuracy_score(data_prediction, y_test))
    return res
dataReal = pd.read_csv("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Программирование и основы алгоритмизации тест.csv", delimiter=";", encoding='cp1251', decimal=",") #delimiter=";",
print(len(data))

not_for_prediction = ["ФИО", "Команда", "Команды", "Направление", "Оценка", "Баллы", "Повышение оценки"]

# cols = dataReal[dataReal.loc[:,:].columns.difference(not_for_prediction)].select_dtypes(exclude=['float']).columns
# dataReal[cols] = dataReal[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
# columns = list(dataReal.columns[pd.Series(dataReal.columns).str.startswith('Посещение ')])
# dataReal[dataReal.loc[:,:].columns.difference(["Пол", "Не сдал(-а)"]+columns+not_for_prediction)] = dataReal[dataReal.loc[:,:].columns.difference(["Пол", "Не сдал(-а)"]+columns+not_for_prediction)].div(100)
# dataReal = dataReal.round(4)

ex = "Контрольная работа 3"
results = pd.DataFrame(index=list(models.keys()), columns=exams)

X = div_100(data[data.loc[:,:ex].columns.difference(not_for_prediction+["Не сдал(-а)"])]) #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
# X[X.loc[:,:].columns.difference(["Пол"])] = X[X.loc[:,:].columns.difference(["Пол"])].div(100)
x_ = div_100(dataReal[dataReal.loc[:,:ex].columns.difference(not_for_prediction+["Не сдал(-а)"])])
Y = data["Не сдал(-а)"]
results = pd.DataFrame(columns=['Directions', "RealPassed"]+list(models.keys()))
results['Directions'] = dataReal["Направление"]
results['RealPassed'] = dataReal["Не сдал(-а)"]

for key, model in models.items():
    model.fit(X, Y)
    data_prediction = model.predict(x_)
    results[key] = data_prediction
    f1 = round(f1_score(dataReal["Не сдал(-а)"], data_prediction), 3)
    print(f"{key}: спрогнозировано; f1 = {f1}")
results.to_excel(f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/ПиОА(сдача) DIV100.xlsx") #, index=False, sep=';', encoding='cp1251')
results

961
k-ближайших соседей: спрогнозировано; f1 = 0.758
Рандомный лес: спрогнозировано; f1 = 0.624
AdaBoostClassifier: спрогнозировано; f1 = 0.793


,Directions,RealPassed,k-ближайших соседей,Рандомный лес,AdaBoostClassifier
0,02.03.03 Математическое обеспечение и админист...,0,0.0,0.0,0.0
1,40.03.01 Юриспруденция,1,1.0,1.0,1.0
2,10.05.01 Компьютерная безопасность,0,0.0,0.0,0.0
3,10.03.01 Информационная безопасность,0,0.0,0.0,1.0
4,10.05.01 Компьютерная безопасность,0,0.0,0.0,0.0
...,...,...,...,...,...
956,16.03.01 Техническая физика,0,0.0,0.0,1.0
957,02.03.03 Математическое обеспечение и админист...,1,0.0,0.0,1.0
958,09.03.02 Информационные системы и технологии,1,1.0,1.0,1.0
959,06.05.01 Биоинженерия и биоинформатика,1,1.0,1.0,1.0


In [ ]:
dataReal = pd.read_csv("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Программирование и основы алгоритмизации тест.csv", delimiter=";", encoding='cp1251', decimal=",") #delimiter=";",
print(len(dataReal))

not_for_prediction = ["ФИО", "Команда", "Команды", "Направление", "Оценка", "Баллы", "Повышение оценки"]

cols = dataReal[dataReal.loc[:,:].columns.difference(not_for_prediction)].select_dtypes(exclude=['float']).columns
dataReal[cols] = dataReal[cols].apply(pd.to_numeric, downcast='float', errors='coerce')

ex = "Контрольная работа 3"
results = pd.DataFrame(index=list(models.keys()), columns=exams)

X = data[data.loc[:,:ex].columns.difference(not_for_prediction+["Не сдал(-а)"])]
x_ = dataReal[dataReal.loc[:,:ex].columns.difference(not_for_prediction+["Не сдал(-а)"])]
Y = data["Не сдал(-а)"]
results = pd.DataFrame(columns=['Directions', "RealPassed"]+list(models.keys()))
results['Directions'] = dataReal["Направление"]
results['RealPassed'] = dataReal["Не сдал(-а)"]
for key, model in models.items():
    model.fit(X, Y)
    data_prediction = model.predict(x_)
    results[key] = data_prediction
    f1 = round(f1_score(dataReal["Не сдал(-а)"], data_prediction), 3)
    print(f"{key}: спрогнозировано; f1 = {f1}")
results.to_excel(f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/ПиОА(сдача) NORMAL.xlsx") #, index=False, sep=';', encoding='cp1251')
results

961
k-ближайших соседей: спрогнозировано; f1 = 0.553
Рандомный лес: спрогнозировано; f1 = 0.598
AdaBoostClassifier: спрогнозировано; f1 = 0.367


,Directions,RealPassed,k-ближайших соседей,Рандомный лес,AdaBoostClassifier
0,02.03.03 Математическое обеспечение и админист...,0.0,0.0,0.0,0.0
1,40.03.01 Юриспруденция,1.0,1.0,1.0,1.0
2,10.05.01 Компьютерная безопасность,0.0,0.0,0.0,0.0
3,10.03.01 Информационная безопасность,0.0,0.0,0.0,0.0
4,10.05.01 Компьютерная безопасность,0.0,0.0,0.0,0.0
...,...,...,...,...,...
956,16.03.01 Техническая физика,0.0,0.0,1.0,0.0
957,02.03.03 Математическое обеспечение и админист...,1.0,0.0,0.0,0.0
958,09.03.02 Информационные системы и технологии,1.0,1.0,0.0,1.0
959,06.05.01 Биоинженерия и биоинформатика,1.0,1.0,1.0,1.0


In [ ]:
dataReal = pd.read_csv("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Программирование и основы алгоритмизации тест.csv", delimiter=";", encoding='cp1251', decimal=",") #delimiter=";",
print(len(dataReal))

not_for_prediction = ["ФИО", "Команда", "Команды", "Направление", "Оценка", "Баллы", "Повышение оценки"]

cols = dataReal[dataReal.loc[:,:].columns.difference(not_for_prediction)].select_dtypes(exclude=['float']).columns
dataReal[cols] = dataReal[cols].apply(pd.to_numeric, downcast='float', errors='coerce')


for ex in exams:
    results = pd.DataFrame(index=list(models_regres.keys()), columns=exams)
    temp_data = data[data.loc[:,:ex].columns]
    temp_data = temp_data.drop_duplicates(keep='last')
    X = div_100(temp_data[temp_data.columns.difference(not_for_prediction+[ex,"Не сдал(-а)"])]) #[name  for name in data.columns.to_list() if ex in name and ex!=name]]
    Y = temp_data[ex] #.to_list().
    x_ = div_100(dataReal[dataReal.loc[:,:ex].columns.difference(not_for_prediction+[ex,"Не сдал(-а)"])])

    results = pd.DataFrame(columns=['Directions', "Real "+ex]+list(models_regres.keys()))
    results['Directions'] = dataReal["Направление"]
    results["Real "+ex] = dataReal[ex].round(2)
    for key, model in models_regres.items():
        model.fit(X, Y)
        data_prediction = model.predict(x_)
        results[key] = data_prediction.round(2)
        f1 = round(mean_absolute_error(dataReal[ex].round(2), data_prediction.round(2)), 3)
        print(f"{key}: спрогнозировано; f1 = {f1}")
    results.to_excel(f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/ПиОА({ex}) DIV100.xlsx") #, index=False, sep=';', encoding='cp1251')

961
k-ближайших соседей: спрогнозировано; f1 = 5.909
Рандомный лес: спрогнозировано; f1 = 5.381
CatBoostRegressor: спрогнозировано; f1 = 5.288
AdaBoostRegressor: спрогнозировано; f1 = 5.767
k-ближайших соседей: спрогнозировано; f1 = 7.706
Рандомный лес: спрогнозировано; f1 = 7.515
CatBoostRegressor: спрогнозировано; f1 = 7.764
AdaBoostRegressor: спрогнозировано; f1 = 7.508
k-ближайших соседей: спрогнозировано; f1 = 7.588
Рандомный лес: спрогнозировано; f1 = 7.107
CatBoostRegressor: спрогнозировано; f1 = 6.81
AdaBoostRegressor: спрогнозировано; f1 = 9.539


In [ ]:
dataReal = pd.read_csv("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Программирование и основы алгоритмизации тест.csv", delimiter=";", encoding='cp1251', decimal=",") #delimiter=";",
print(len(dataReal))

not_for_prediction = ["ФИО", "Команда", "Команды", "Направление", "Оценка", "Баллы", "Повышение оценки"]

cols = dataReal[dataReal.loc[:,:].columns.difference(not_for_prediction)].select_dtypes(exclude=['float']).columns
dataReal[cols] = dataReal[cols].apply(pd.to_numeric, downcast='float', errors='coerce')


for ex in exams:
    results = pd.DataFrame(index=list(models_regres.keys()), columns=exams)
    temp_data = data[data.loc[:,:ex].columns]
    temp_data = temp_data.drop_duplicates(keep='last')
    X = temp_data[temp_data.columns.difference(not_for_prediction+[ex,"Не сдал(-а)"])]
    Y = temp_data[ex]
    x_ = dataReal[dataReal.loc[:,:ex].columns.difference(not_for_prediction+[ex,"Не сдал(-а)"])]

    results = pd.DataFrame(columns=['Directions', "Real "+ex]+list(models_regres.keys()))
    results['Directions'] = dataReal["Направление"]
    results["Real "+ex] = dataReal[ex].round(2)
    for key, model in models_regres.items():
        model.fit(X, Y)
        data_prediction = model.predict(x_)
        results[key] = data_prediction.round(2)
        f1 = round(mean_absolute_error(dataReal[ex].round(2), data_prediction.round(2)), 3)
        print(f"{key}: спрогнозировано; f1 = {f1}")
    results.to_excel(f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/ПиОА({ex}) NORMAL.xlsx") #, index=False, sep=';', encoding='cp1251')

961
k-ближайших соседей: спрогнозировано; f1 = 4.279
Рандомный лес: спрогнозировано; f1 = 5.223
CatBoostRegressor: спрогнозировано; f1 = 5.288
AdaBoostRegressor: спрогнозировано; f1 = 5.053
k-ближайших соседей: спрогнозировано; f1 = 6.921
Рандомный лес: спрогнозировано; f1 = 7.295
CatBoostRegressor: спрогнозировано; f1 = 7.761
AdaBoostRegressor: спрогнозировано; f1 = 7.618
k-ближайших соседей: спрогнозировано; f1 = 7.043
Рандомный лес: спрогнозировано; f1 = 7.293
CatBoostRegressor: спрогнозировано; f1 = 6.811
AdaBoostRegressor: спрогнозировано; f1 = 9.626
